In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### First, we load the titatic dataset and convert all categorical variables to one-hot

In [ ]:
data_frame = pd.read_csv('titanic.csv')
data_frame = data_frame.drop(['Name', 'Ticket', 'Cabin'], axis=1)
data_frame = pd.get_dummies(data_frame, columns=[...])
data_frame = data_frame.fillna(data_frame.mean())
data_frame

### Next, we convert the dataset to a NumPy array and normalize it

In [ ]:
labels = data_frame[['Survived']].to_numpy().astype(float)
observ = data_frame.drop(['PassengerId', 'Survived'], axis=1).to_numpy().astype(float)
X_train, X_test, y_train, y_test = train_test_split(observ, labels, test_size=0.25, random_state=42)

In [ ]:
# We have to normalize Age and Fare, so columns 0,3
age_min, age_max = ...
fare_min, fare_max = ...
# Apply
X_train[:,0] = (X_train[:,0] - age_min) / (age_max - age_min + 1e-5)
X_test[:,0] = (X_test[:,0] - age_min) / (age_max - age_min + 1e-5)
X_train[:,3] = (X_train[:,3] - fare_min) / (fare_max - fare_min + 1e-5)
X_test[:,3] = (X_test[:,3] - fare_min) / (fare_max - fare_min + 1e-5)

### Now, let's create our sequential model with four linear layers

In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
first_model = nn.Sequential(
    nn.Linear(..., 256),
    nn.ReLU(),
    nn.Dropout(),
    
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(),
    
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(),
    
    nn.Linear(64, 1),
    nn.Sigmoid()
)

### We can already apply it to our test dataset

In [ ]:
first_model(torch.tensor(X_test[:20], dtype=torch.float))

### Now, let's fit our model

In [ ]:
first_model.fit(X_train, y_train, X_test, y_test)

### As you may have seen, this does not work. We have to implement the training ourselves!

In [ ]:
class MyMLPModel(nn.Module):
    def __init__(self, input, *hidden_layers, lr=0.1, dropout=0.2):
        super().__init__() # <- Very important!
        self.lr = lr
        ## Build model
        n_neurons = [input] + list(hidden_layers)
        self.layers = []
        for i, o in zip(n_neurons[:-1], n_neurons[1:]):
            self.layers += [
                ...
            ]
        self.layers += [
            ...
        ]
        
        self.layers = nn.Sequential(*self.layers) # Create a sequential model
        
    def forward(self, X):
        return self.layers(X)
    
    def predict(self, X, th=0.5):
        X = torch.tensor(X, dtype=torch.float)
        with torch.no_grad():
            y_hat = ...
        return (...).float()
    
    def train_step(self, X, y):
        y_hat = self(X)
        return ...
        
    def validation_step(self, X, y):
        with torch.no_grad():
            return ...
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.lr)
        return optimizer
        
    def fit(self, X_train, y_train, X_valid, y_valid, epochs=10):
        ## Convert dataset
        X_train = torch.tensor(X_train, dtype=torch.float)
        y_train = torch.tensor(y_train, dtype=torch.float)
        
        X_valid = torch.tensor(X_valid, dtype=torch.float)
        y_valid = torch.tensor(y_valid, dtype=torch.float)
        
        ## Load Optimizer
        optimizer = self.configure_optimizers()
        
        for epoch in range(epochs):
            print(f'{epoch+1}/{epochs}:')
            # Training
            self.train() # Set model to training mode
            ... # Sets all gradients to Zero
            loss = ...# Execute Forward pass and calculate Loss
            ... # Execute Backward pass
            ... # Update weights
            self.eval() # Set model to validation mode
            
            # Validation
            loss_valid = ...
            print(f'Training Loss: {loss.item():1.4f}', f'Validation Loss: {loss_valid.item():1.4f}')
            
        return self

### Create a model similar to the one before

In [ ]:
second_model = MyMLPModel(X_train.shape[1], 32, 16, 8, 4)
second_model

In [ ]:
second_model.predict(X_test[:20])

### Train it and calculate the test accuracy 

In [ ]:
second_model = second_model.fit(X_train, y_train, X_test, y_test, epochs=1000)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = second_model.predict(X_test)
accuracy_score(y_test, y_pred)